In [1]:
%matplotlib inline
import sys
import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# The GPU id to use, usually either "0" or "1"
#os.environ["CUDA_VISIBLE_DEVICES"]="0" 
import argparse
from skimage import io
from keras.models import load_model
sys.path.append("../")
sys.path.append("../")
from utils.load_data import *
from utils.paths import *
from utils.utils import *
from utils.metadata import *
from utils.dataflows import *
from utils.generator import *
from networks.networks import *
from networks.networks import *
from keras.layers import Input
from keras.models import Model

Using TensorFlow backend.


In [2]:
mod = load_model("../models/body_sept_alex_net64_margin2_reg.h5", compile=False)

In [3]:
mod.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 320, 250, 3)  0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 320, 250, 3)  0                                            
__________________________________________________________________________________________________
sequential_1 (Sequential)       (None, 64)           31755904    input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 1)            0           sequential_1[1][0]               
          

In [4]:
OUTPUT_PATH = "../stats/"

OUTPUT_PATH += 'body_sept_alex_net64_margin2_reg'

WEIGHTS_FILE = os.path.join(WEIGHTS_DIR, "body_sept_alex_net64_margin2_reg-w.h5")

In [2]:
train_data = pd.read_csv("../datasets/body_sept/train_pairs.csv")
test_data = pd.read_csv("../datasets/body_sept/test_pairs.csv")

In [3]:
X1 = train_data.X1[:25]
X2 = train_data.X2[:25]

X1_test = test_data.X1[:25]
X2_test = test_data.X2[:25]

In [4]:
X1_gen = TestGenerator(X1)
X2_gen = TestGenerator(X2)

X1_t_gen = TestGenerator(X1_test)
X2_t_gen = TestGenerator(X2_test)

In [5]:
X1_gen[0]

array([[[[ 63, 138, 170],
         [ 64, 139, 171],
         [ 64, 139, 171],
         ..., 
         [ 66, 139, 172],
         [ 65, 138, 171],
         [ 65, 138, 171]],

        [[ 64, 139, 171],
         [ 64, 139, 171],
         [ 64, 139, 171],
         ..., 
         [ 66, 139, 172],
         [ 65, 138, 171],
         [ 65, 138, 171]],

        [[ 64, 139, 171],
         [ 64, 139, 171],
         [ 64, 139, 171],
         ..., 
         [ 66, 139, 172],
         [ 65, 138, 171],
         [ 65, 138, 171]],

        ..., 
        [[ 79, 150, 182],
         [ 79, 150, 182],
         [ 79, 150, 182],
         ..., 
         [ 77, 146, 177],
         [ 77, 146, 177],
         [ 77, 146, 177]],

        [[ 79, 150, 182],
         [ 79, 150, 182],
         [ 79, 150, 182],
         ..., 
         [ 77, 146, 177],
         [ 77, 146, 177],
         [ 77, 146, 177]],

        [[ 79, 150, 182],
         [ 79, 150, 182],
         [ 79, 150, 182],
         ..., 
         [ 77, 146, 177],
  

In [7]:
# Load Model
#######################################
print("Loading Model...")
network = base_network('alex_net64_reg', [320, 250, 3])

network_input = Input([320, 250, 3])

model = network(network_input)

model = Model(network_input, model)
model.summary()
model.load_weights(WEIGHTS_FILE)
#######################################

Loading Model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 320, 250, 3)       0         
_________________________________________________________________
sequential_2 (Sequential)    (None, 64)                31755904  
Total params: 31,755,904
Trainable params: 31,755,264
Non-trainable params: 640
_________________________________________________________________


In [9]:
len(X1_gen), len(X1)

(458397, 458397)

In [ ]:
# Predict on training and testing data
print("Predicting Training Data...")
train_pred_x1 = model.predict_generator(X1_gen)
train_pred_x2 = model.predict_generator(X2_gen)

print("Predicting Testing Data...")
test_pred_x1 = model.predict_generator(X1_t_gen)
test_pred_x2 = model.predict_generator(X2_t_gen)

Predicting Training Data...


In [12]:
X1[0:15].values

array(['../datasets/body_sept/train_data/frame/001356/Bee-001356-000245.jpg',
       '../datasets/body_sept/train_data/frame/007325/Bee-007325-001470.jpg',
       '../datasets/body_sept/train_data/frame/007673/Bee-007673-001193.jpg',
       '../datasets/body_sept/train_data/track/001719/Bee-008899-001719.jpg',
       '../datasets/body_sept/train_data/frame/008181/Bee-008181-001607.jpg',
       '../datasets/body_sept/train_data/track/001630/Bee-008290-001630.jpg',
       '../datasets/body_sept/train_data/frame/000091/Bee-000091-000025.jpg',
       '../datasets/body_sept/train_data/frame/003035/Bee-003035-000271.jpg',
       '../datasets/body_sept/train_data/track/001610/Bee-008181-001610.jpg',
       '../datasets/body_sept/train_data/track/000689/Bee-003084-000689.jpg',
       '../datasets/body_sept/train_data/track/001493/Bee-007642-001493.jpg',
       '../datasets/body_sept/train_data/track/000351/Bee-001716-000351.jpg',
       '../datasets/body_sept/train_data/frame/008651/Bee-008651

In [7]:
output_dim = 64 #model.output.shape.as_list()[1]

In [8]:
size = len(X1)
batch = 10

x1_emb = np.empty((0,output_dim))
x2_emb =  np.empty((0,output_dim))
for i in range(0, size, batch):
    if i + batch >= size:
        x1_batch = X1[i:].values
        x2_batch = X2[i:].values
    else:   
        x1_batch = X1[i:i+batch].values
        x2_batch = X2[i:i+batch].values
    x1_imgs = np.array(io.imread_collection(x1_batch))/255.
    x2_imgs = np.array(io.imread_collection(x2_batch))/255.
    #x1_emb = np.append(x1_emb, model.predict(x1_imgs), axis=0)
    #x2_emb = np.append(x2_emb, model.predict(x2_imgs), axis=0)
    print(i)
    

0
10
20


In [9]:
x1_imgs[0]

array([[[ 48, 109, 140],
        [ 49, 110, 141],
        [ 50, 111, 142],
        ..., 
        [ 70, 143, 176],
        [ 67, 140, 173],
        [ 67, 140, 173]],

       [[ 48, 109, 140],
        [ 48, 109, 140],
        [ 49, 110, 141],
        ..., 
        [ 70, 143, 176],
        [ 67, 140, 173],
        [ 67, 140, 173]],

       [[ 48, 109, 140],
        [ 48, 109, 140],
        [ 49, 110, 141],
        ..., 
        [ 71, 144, 177],
        [ 67, 140, 173],
        [ 67, 140, 173]],

       ..., 
       [[ 49, 105, 132],
        [ 49, 105, 132],
        [ 50, 106, 133],
        ..., 
        [ 61, 130, 159],
        [ 60, 129, 158],
        [ 60, 129, 158]],

       [[ 49, 105, 132],
        [ 49, 105, 132],
        [ 50, 106, 133],
        ..., 
        [ 61, 130, 159],
        [ 61, 130, 159],
        [ 61, 130, 159]],

       [[ 49, 105, 132],
        [ 49, 105, 132],
        [ 50, 106, 133],
        ..., 
        [ 60, 129, 158],
        [ 62, 131, 160],
        [ 62, 131,

In [34]:
train_dist = distance(x1_emb , x2_emb) 
#test_dist = distance(test_pred_x1, test_pred_x2) 

In [35]:
train_dist.shape, train_data.y.shape, X1.shape

((25,), (458397,), (25,))

In [15]:
test_dist.shape, test_data.y.shape, X1_test.shape

((164352,), (164434,), (164434,))

In [28]:
len(X1_gen)

3581

In [18]:
##########################################################
# Save Train and Test predictions
# TODO: Refactor this code
df_dict_train = {
    "X1": X1,
    "X2": X2,
    "y_pred": train_dist,
    "y_true": train_data.y
}

df_dict_test = {
    "X1": X1_test,
    "X2": X2_test,
    "y_pred": test_dist,
    "y_true": test_data.y
}

In [19]:
output_name = 'body_sept_alex_net64_margin2_reg'
train_csv_filename = OUTPUT_PATH + "/{0}-train_results.csv".format(output_name)
test_csv_filename = OUTPUT_PATH + "/{0}-test_results.csv".format(output_name)

In [20]:
train_df = pd.DataFrame.from_dict(df_dict_train)
train_df["training"] = 1
print("Saving train csv in {}".format(train_csv_filename))
train_df.to_csv(train_csv_filename, index=False)

ValueError: array length 458368 does not match index length 458397

In [ ]:
test_df = pd.DataFrame.from_dict(df_dict_test)
test_df["training"] = 0
print("Saving test csv in {}".format(test_csv_filename))
test_df.to_csv(test_csv_filename, index=False)